## Grid Search CV

GridSearchCV es una herramienta de la libreria Sklearn.

Esta herramienta hace una busqueda exhaustiva (o por fuerza bruta) de los mejores parámetros de un modelo siguiendo alguna métrica en particular.

_**Documentacion:** https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html_

In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Normalizacion
from sklearn.preprocessing import MinMaxScaler

# Train, Test
from sklearn.model_selection import train_test_split

# Metricas
from sklearn.metrics import jaccard_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [16]:
# Dataset del titanic preprocesado en clase

titanic = pd.read_csv("titanic_preprocesamiento.csv")

X = titanic.drop(["Fare-Binning", "Age-Binning", "Survived"], axis = 1)
y = titanic["Survived"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 42)

print(f"X_train: {X_train.shape}, y_train: {y_train.shape}")
print(f"X_test: {X_test.shape},  y_test: {y_test.shape}")

X_train: (621, 11), y_train: (621,)
X_test: (267, 11),  y_test: (267,)


### Modelo

In [17]:
from sklearn.neighbors import KNeighborsClassifier

In [18]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

### Predicciones

In [19]:
yhat = model.predict(X_test)

yhat

array([0., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
       0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1.,
       0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1.,
       0., 1., 0., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1.,
       0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 0., 0., 1., 0., 1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1.,
       0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
       0., 1., 1., 0., 0.

In [20]:
print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
print("Accuracy:"     , accuracy_score(y_test, yhat))
print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
print("ROC AUC:"      , roc_auc_score(y_test, yhat))

Jaccard Index: 0.5577427821522309
Accuracy: 0.7378277153558053
Precisión: 0.7385509227614491
Sensibilidad: 0.7038551401869159
F1-score: 0.7097826086956521
ROC AUC: 0.7038551401869159


### GridSearchCV

In [21]:
from sklearn.model_selection import GridSearchCV

In [22]:
%%time

# Modelo
model = KNeighborsClassifier()

# Parametros a iterar
params = {"n_neighbors" : [3, 4, 5, 6]}

# Metricas
scorers = {"f1_macro", "accuracy", "recall_macro"}

#GridSearchCV
grid_solver = GridSearchCV(estimator  = model     , 
                           param_grid = params    , 
                           scoring    = scorers   ,
                           cv         = 5         ,
                           refit      = "accuracy",
                           n_jobs     = -1         )

# Resultados
model_result = grid_solver.fit(X_train, y_train)

Wall time: 19.3 s


In [23]:
# model_result.best_estimator es el mejor modelo que obtuvimos al iterar sobre todos los parámetros
# 

model_result.best_estimator_

KNeighborsClassifier(n_neighbors=3)

In [24]:
yhat = model_result.best_estimator_.predict(X_test)

print("Jaccard Index:", jaccard_score(y_test, yhat, average = "macro"))
print("Accuracy:"     , accuracy_score(y_test, yhat))
print("Precisión:"    , precision_score(y_test, yhat, average = "macro"))
print("Sensibilidad:" , recall_score(y_test, yhat, average = "macro"))
print("F1-score:"     , f1_score(y_test, yhat, average = "macro"))
print("ROC AUC:"      , roc_auc_score(y_test, yhat))

Jaccard Index: 0.5467328974761678
Accuracy: 0.7265917602996255
Precisión: 0.7212261041529334
Sensibilidad: 0.6960280373831775
F1-score: 0.7010812054290314
ROC AUC: 0.6960280373831775


In [25]:
model_result.cv_results_

{'mean_fit_time': array([0.01977735, 0.01910071, 0.03299823, 0.03027644]),
 'std_fit_time': array([0.00401813, 0.00249508, 0.00790109, 0.01007698]),
 'mean_score_time': array([0.07749052, 0.06152864, 0.06420865, 0.06994252]),
 'std_score_time': array([0.02763795, 0.00315168, 0.01229118, 0.00642498]),
 'param_n_neighbors': masked_array(data=[3, 4, 5, 6],
              mask=[False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 3},
  {'n_neighbors': 4},
  {'n_neighbors': 5},
  {'n_neighbors': 6}],
 'split0_test_f1_macro': array([0.67023173, 0.6352887 , 0.70911783, 0.69287469]),
 'split1_test_f1_macro': array([0.67619685, 0.63777994, 0.66451613, 0.63995354]),
 'split2_test_f1_macro': array([0.7158276 , 0.67968689, 0.68205128, 0.66027397]),
 'split3_test_f1_macro': array([0.63011364, 0.63404784, 0.60582539, 0.62594268]),
 'split4_test_f1_macro': array([0.64687842, 0.58202247, 0.59058283, 0.58262231]),
 'mean_test_f1_macro': array([0.66

In [26]:
print(model_result.cv_results_["mean_test_recall_macro"].mean())
print(model_result.cv_results_["mean_test_f1_macro"].mean())
print(model_result.cv_results_["mean_test_accuracy"].mean())

print(model_result.best_score_)
print(model_result.best_params_)

0.6455908799803342
0.6480917374876254
0.6927903225806451
0.6972516129032258
{'n_neighbors': 3}


In [ ]:
%%time

from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

params = {"n_estimators"           : [100, 200, 300], # Numero de arboles
          "criterion"              : ["gini", "entropy"], # Es la función para medir la calidad de una división/split.
          "max_depth"              : [3, 4, 5], # La profundidad máxima del árbol.
          "max_features"           : [2, 3], # El número de características (atributos) a considerar en cada split
          "max_leaf_nodes"         : [8], # Maximo de nodos hoja del arbol
          "min_impurity_decrease"  : [0.02, 0.3], # Un nodo se dividirá si esta división induce una disminución de la impureza mayor o igual a este valor.
          "min_samples_split"      : [2, 5]} # El número mínimo de muestras requeridas para llegar a nodo hoja.

scorers = {"f1_macro", "accuracy", "recall_macro"}

grid_solver = GridSearchCV(estimator  = model    , 
                           param_grid = params   , 
                           scoring    = scorers  ,
                           cv         = 5        ,
                           refit      = "accuracy",
                           n_jobs     = -1        )

model_result = grid_solver.fit(X_train, y_train)

print(model_result.cv_results_["mean_test_recall_macro"].mean())
print(model_result.cv_results_["mean_test_f1_macro"].mean())
print(model_result.cv_results_["mean_test_accuracy"].mean())

print("*"*100)

print(model_result.best_score_)
print(model_result.best_params_)

In [ ]:
# Metricas para el GridSearchCV

from sklearn.metrics import SCORERS

sorted(SCORERS.keys())